Dear user please run each cell manually until alerta alerta. 


This one only is about pkPDB


In [2]:
from Bio.PDB import PDBList
from Bio.PDB import *
import pandas as pd
import MDAnalysis as mda
from MDAnalysis import Merge
import numpy as np
np.set_printoptions(threshold=1000) #1000 is default
import random
import math
from IPython.display import display, HTML
import re
import os
#import nglview as nv
from ase import Atoms
from ase import Atom

In [3]:
local_folder="/Users/jessihoernschemeyer/pKaSchNet"
pkPDB_CSV = f"{local_folder}/pkas.csv"
#PKAD_CSV = f"{local_folder}/WT_pka.csv"
rtf = f"{local_folder}/rtf/top_all36_prot.rtf"

In [4]:
def input_df_pkPDB(csv_file): #pypka/pkPDB dataframe production
    """input_pkPDB_df imports the CSV file from pkPDB and makes a pandas dataframe, henceforth referred to as 'input df'."""
    ID=[]
    df = pd.read_csv(csv_file, nrows=3000, delimiter=";") [["idcode","residue_name","residue_number", "chain","pk"]] #nrows loads only the first 100 rows because its so big. remove when doing all data

#make a unique identifier ('ID' array) and add it as column to df.
    for i in range(len(df)):
        pdb=str(df.iloc[i]['idcode'])
        res_n=str(df.iloc[i]['residue_number'])
        res=str(df.iloc[i]['residue_name'])
        chain=str(df.iloc[i]['chain'])
        ID = np.append(ID,("-".join([pdb,res,res_n, chain])))
    df['ID'] = ID
#clean up df: rename columns to match df from pkad, sort by PDB ID first and then Res ID and then make index start from 1. 
    df = df.rename(columns={'idcode': 'PDB ID', 'residue_number': 'Res ID', 'residue_name': 'Res Name', 'residue_number': 'Res ID', 'pk': 'pKa', 'chain' : 'Chain'}) #rename columns to match df from pkad 
    df.sort_values(['PDB ID', 'Res ID'], ascending=[True, True], inplace=True) #sorts both
    df = df.reset_index(drop=True) #index start from 1 again
        
    return df    
inputdf_pkPDB=input_df_pkPDB(pkPDB_CSV) #inputdf_pkPDB #display(HTML(inputdf_pkPDB.to_html()))

FileNotFoundError: [Errno 2] No such file or directory: '/Users/jessihoernschemeyer/pKaSchNet/pkas.csv'

In [69]:
def download_PDB(pdbname, input_df, iterations): 
    """downloads PDBs from biopython. iterations = the number is the number of pdbs we make neighborhoods for """
    pdbl = PDBList()
    if os.path.isfile(f"{local_folder}/PDB/pdb{pdbname}.ent") == False: #download pdb if we dont have the file already
        pdbl.retrieve_pdb_file(pdbname,obsolete=False, pdir='PDB',file_format = 'pdb')
    else: #download obsolete entries
        pdbl.retrieve_pdb_file(pdbname,obsolete=True, pdir='PDB',file_format = 'pdb')
    try: #import with mdanalysis
        u = mda.Universe(f"{local_folder}/PDB/pdb{pdbname}.ent") #pdb_file = f"{local_folder}/PDB/pdb{pdbname}.ent"
    except:
        print(f"the pdb file {pdbname} doesnt exist")
        pass
    return u #universe

In [70]:
def titratable_res_info(resid_chain_in, u):
    """gets residue name, atom names, residue number, and xyz positions for the titratable residue
    u = MDAuniverse"""
    ASE_atomnames, ASE_positions, x_tit, y_tit, z_tit=[],[],[], [], []
    
    res_atoms=u.select_atoms(resid_chain_in) #make universe for atoms of titratable residue 
    atomname_tit = res_atoms.names #atom names of titratable residue
    ASE_atomnames.append(atomname_tit) #ASE output
    positions_tit=res_atoms.positions  #atomic positions of titratable residue 
    ASE_positions.append(positions_tit) #ASE output
    res_name_tit = res_atoms.resnames #residue name (it is the same for each atom.)
    res_ids_tit = np.repeat(res_atoms.residues.resids, len(res_name_tit)) #make long array of the res number of the titratable residue 
    for j in range(len(positions_tit)): #x, y, and z coordinate for each atom of 1 residue
        x_tit=np.append(x_tit,positions_tit[j,0]) 
        y_tit=np.append(y_tit,positions_tit[j,1])
        z_tit=np.append(z_tit,positions_tit[j,2])
        
    return res_name_tit, res_ids_tit, atomname_tit, x_tit, y_tit, z_tit, ASE_atomnames, ASE_positions

In [71]:
#called/used in 'neighborhood'
def MDAvicinity(in_cutoff, resid_chain_in, u, res_name_tit, res_ids_tit, atomname_tit, x_tit, y_tit, z_tit, ASE_atomnames, ASE_positions): 
    """generates the mda universe containing the residues neighboring the titratable residue for the function 'neighborhoods'."""
    #collects atoms w/in a cutoff distance of the titratable res which arent part of H20 molecules or residues w/ numbers &...
    nahe_one=((((u.select_atoms(f'around {in_cutoff} {resid_chain_in}')).select_atoms('not resname HOH')).select_atoms('resname [!0-9]**')).select_atoms('resname *[!0-9]*')).select_atoms('resname **[!0-9]')
    residues_inner = nahe_one.residues
    res_name_inner = np.append(res_name_tit,residues_inner.atoms.resnames) #vicinity residues' names 
    res_ids_inner = np.append(res_ids_tit, residues_inner.atoms.resids) #vicinity residues' residue number
    atomname_inner = np.append(atomname_tit, residues_inner.atoms.names) #atom NAMES of atoms compromising each vicinity residue 
    ASE_atomnames.append(atomname_inner) #output for ASE  
    positions_inner=residues_inner.atoms.positions #generates positions of each atom w/in each vicinity residue & seperates them into coordinate arrays
    ASE_positions.append(positions_inner) #output for ASE
    for j in range(len(positions_inner)): #x, y, and z coordinate for each atom of 1 residue #VICINITY XYZ
        x_tit=np.append(x_tit,positions_inner[j,0])
        y_tit=np.append(y_tit,positions_inner[j,1])
        z_tit=np.append(z_tit,positions_inner[j,2])
        x_vic_tit, y_vic_tit, z_vic_tit = x_tit, y_tit, z_tit
    
    return res_name_inner, res_ids_inner, atomname_inner, x_vic_tit, y_vic_tit, z_vic_tit, ASE_positions

In [76]:
def neighborhoods(input_df, in_cutoff):  
    #ASE_atomnames is for a single residue, ASE_names is the complete list for all residues
    """neighborhoods extracts the residue number and PDB name from the input dataframe and downloads the PDB. It then 
    generates a MDA mini universe consisting of the atoms belonging to residues neighboring the titratable residue of
    focus using the latter's residue number. From that, the atomic coordinates and atom names are generated and stored 
    in a new dataframe (along with the pdb-id, res-id, pKa-value, and residue-name from the input df) which corresponds 
    to 1 row o  `f the input dataframe, named according to the row they belong to (=column name df's first column).
    
    The locally downloaded PDB file is deleted after processing. Every tenth mini universe is saved as a PDB.
    
        Args:
        input_df (PANDAS df):    Information from the CSVs downloaded from the pKa value databases PKAD and pkPDB.
        in_cutoff (float):        Cutoff distance which specifies the size of the titratable residue's neighborhood
       
    Returns:
        df_list (list): a list of generated dfs. Each df corresponds to 1 residue (specified by res #) from 1 pdb, alphabetically 
        sorted by residue name.
        
        EACH DF IN A MINI BATCH CORRESPONDS TO PDB ID - RES NAME - RES NUMBER WHERE RES NUMBER IS THE UNIQUE ELEMENT
        
        ASE_atomnames (list of strings): list of atom names of the titratable residue we are zooming into (ASE)
        ASE_positions (list of arrays): positions of atoms of the titratable residue we are zooming into (ASE)"""
    pdbl = PDBList() #--initialization center-- #load biopython pdb list
    df_list, ASE_names, ASE_pos = [], [], []
    for i in range(1): #range(len(input_df)) #the number is the number of pdbs we make neighborhoods for.
        pdbname=str.lower(input_df.iloc[i]['PDB ID']) #Extracting PDB name 
        u = download_PDB(pdbname, input_df, 10) #10 is  # iternations 
        if bool(isinstance((input_df.iloc[1]['Res ID']), float) == True): #get residue number 
            val=math.trunc(input_df.iloc[i]['Res ID'])      
        else:
            val=input_df.iloc[i]['Res ID'] 
        resid_chain_in=f"resid {val} and segid {input_df.iloc[i]['Chain']}"   #atom selection string
        res_name_tit, res_ids_tit, atomname_tit, x_tit, y_tit, z_tit, ASE_atomnames, ASE_positions = titratable_res_info(resid_chain_in, u) #Get info for titratable residue
        res_name_inner, res_ids_inner, atomname_inner, x_vic_tit, y_vic_tit, z_vic_tit, ASE_positions = MDAvicinity(in_cutoff, resid_chain_in, u, res_name_tit, res_ids_tit, atomname_tit, x_tit, y_tit, z_tit, ASE_atomnames, ASE_positions)
        ASE_names.append(ASE_atomnames) 
        ASE_pos.append(ASE_positions)  
        df = pd.DataFrame({"PDB": pdbname,
                    "Res Name": res_name_inner,
                    "Res No": res_ids_inner,
                    "Atom Name": atomname_inner,
                    "x": x_vic_tit,
                    "y":y_vic_tit,
                    "z":z_vic_tit
                          })
        df = df.reset_index(drop=True)
        df_list.append(df.rename(columns={'Res Name': 'Res_Name'})) #add each df, which represents the information about the neighborhood of the titratable residue, to a list 
        df.index.name = str((input_df.iloc[i]['ID'])).lower() #makes the index column name of each df (mini batch) = the identifer for which the neighborhood information belongs to   #UNIQUE ID
        #if i + 1 < len(input_df) -1: AUTOMATICALLY DELETE PDBS HERE
            #if input_df.iloc[i]['PDB ID'] != input_df.iloc[i+1]['PDB ID']:
                #os.remove(f"{local_folder}/PDB/pdb{pdbname}.ent")
            #else: 
                #pass
    return df_list, ASE_names, ASE_pos
dflist, ASE_names, ASE_pos = neighborhoods(inputdf_pkPDB, 9)

Structure exists: 'PDB/pdb155c.ent' 


In [73]:
def dictionary(df, df2): #df = original df #df2=list of dfs
    IDkeys, dicts_dfs, leng =[], [], len(df)
    for i in range(leng):     #get the KEYS, which are the unique ID: pdb-res_num-res_name
        IDkeys.append(str.lower(df.iloc[i]['ID']))
        IDkeys=IDkeys[0:len(df2)]         
    for j in range(len(IDkeys)):
        dictt  = { IDkeys[j] : df2[j] for j in range(len(IDkeys)) }
    return dictt

In [74]:
def rtf_parse_charges(rtf, name):
        charges=[]
        with open(rtf, 'r') as file:
                text = file.read() 
        index1 = text.find(f"RESI {name}") #name is residue we are seeking 
        index2 = text.find("BOND",index1) #stops parsing at "bond" which is where partial charges stop
        string=text[index1:index2] #long string
        list_str=re.split(r'\s',string) #turn the long string into a list of strings
        list_str = [x for x in list_str if x.strip()] #remove empty entries
        while len(list_str) >= 4: #3 is the size the information i am extracting from the list of strings
                if list_str.count("ATOM") == 0:
                        break
                index = list_str.index("ATOM")
                charge = list_str[index+1:index+4] #gets the atom name, atom type, and partial charge 
                charges.append(charge) #charges is list of each atoms charge
                list_str=list_str[index+4:len(string)] #delete the charge which was just stored, from the list we are extracting info from
        return charges #rtf_parse_charges(rtf, "ALA") 

In [75]:
def partial_charges(rtf, df_list, og_df): #og_df is input for the mapping using dictionary().
    """partial_charges parses an rtf file containing the topology of the residues within the mini batch data frame and
    extracts the partial charge and atom type from it. Then, the atom charge and type information is stored in the 
    mini batch data frame.
    
    args: 
        rtf (string) : path to rtf file 
        df_list (list) : list of dataframes, each of which is a mini batch. OUTPUT OF NEIGHBORHOODS
        og_df = original df
    returns:
        df_list_final (list) : list of dataframes, each of which is the input mini batch with the partial charge and 
                           atom type information added."""                    
    df_list_final, n =[], []
    for j in range(len(df_list)):    #len(df_list)
        n, n_dup = [], 0 #sum(n) will be the number that we skip by in the next pass
        df=(df_list[j]).copy()  #get the neighborhood df
        len_old=len(df)
        reslist=list(df['Res_Name']) #i make this to generate the last name
        try:
            final_name=df.Res_Name[len(df)-1] #gets the res name of the last entry in the neighborhood df
        except:
            final_n=len(reslist) #stops this loop at the last titratable residue
            continue
        for i in range(len(df)):
            keys, types, partials = [], [], []
            name=df.Res_Name[i] #we are iterating down the list of resnames. gets the res name
            charges=rtf_parse_charges(rtf, name) #charges = [[atom name, atom type, charge], charge2]. charges is going down the neighborhoods df. thus, there will be repeats
            dict_types, dict_partials, keys, types, partials = {},{}, [], [], []
            for i in range(len(charges)): #make the keys, aka the atom names
                keys.append(charges[i][0])
            for i in range(len(charges)): #get the atom type (vals for dic 1)
                types.append(charges[i][1])
            for i in range(len(charges)): #get partial charge (vals for dic 2)
                partials.append(charges[i][2])
            #make the dictionaries, "unique ID" is dict key
            dict_types  = { keys[j] : types[j] for j in range(len(keys)) }
            dict_partials = { keys[j] : partials[j] for j in range(len(keys)) } #bad charm usually goes below this line
            #mapping to atom name in pandas
            df.loc[df.Res_Name==name, 'Atom Type'] = df["Atom Name"].map(dict_types)
            df.loc[df.Res_Name==name, 'Partial Charge'] = df["Atom Name"].map(dict_partials)  
        if name == final_name or len(set(reslist))==1:
            df_list_final.append(df)                           
#probably we will delete this IM CALLING "bad charm" so i wont bother cleaning up
            if keys.count("CD") > 0: # the one used in RTF #charm uses cd for atom name cd1 so i need to add a dict entry
                cd_type=dict_types["CD"] #retrieve value
                cd_charge=dict_partials["CD"]
                dict_partials["CD1"]=cd_charge #add entry to dict #THE ONE USED IN MY CODE
                dict_types["CD1"]=cd_type
            if keys.count("HN") > 0: 
                cd_type=dict_types["HN"] #retrieve value
                cd_charge=dict_partials["HN"]
                dict_partials["H"]=cd_charge #add entry to dict
                dict_types["H"]=cd_type
            if keys.count("OXT") > 0: 
                dict_partials["OXT"]=-0.51
                dict_types["OXT"]="O" #oxt_charge.... cd_type
            if keys.count("H") > 0:
                cd_type=dict_types["H"] #retrieve value
                cd_charge=dict_partials["H"] 
                dict_partials["HB3"]=cd_charge #add entry to dict #THE ONE USED IN MY CODE
                dict_types["HB3"]=cd_type #if name.... ) went here b4                       
    dictt=dictionary(og_df, df_list_final)

    return dictt
dictt=partial_charges(rtf, dflist, inputdf_pkPDB)

*^*^*^*^*^*^*^*^ alerta alerta! Ignore everything below  *^*^*^*^*^*^*^*^*^

20 lines of code exlcuding comments, doc strings. loop is single block w no breaks. if u think need a break, then make a function there

TODO: estimate size, chain x?


In [ ]:
def check_NaN():
    resnamen=[]
    for i in range(len(dflist)): #len(dflist)
        df = dictt[dflist[i].index.name]
        if df['Partial Charge'].isnull().values.any() == True:#checks for any NaN
            indices_list = (df['Partial Charge'].isnull()).to_list() #Produces list with indices of all NaNs
            for position, item in enumerate(indices_list): #getting indices of rows with missing partial charges
                if item == 1:
                    name = df.iloc[position]['Res_Name']
                    resnamen.append(df.iloc[position]['Res_Name'])
                    missing_res=set(resnamen)
    return missing_res

In [ ]:
def ASE(ASE_names, ASE_pos, dictt): #dont focus on this
    """input: list of lists, where each constitutant list is the atom names of the atoms in a residue.
    : list of arrays, where each array is all of the positions of a residue (residue indicated by index)"""
    one_res_atoms, list_Atoms_objs, charges =[], [], []
    for j in range(1):     #for j in range(len(dflist)): # all of the proteins
        pos=np.concatenate((ASE_pos[j][0], ASE_pos[j][1]))
        for k in range(228):         #for k in range(len(pos)):
            charge=dictt[dflist[0].index.name][0:228].iloc[k]['Partial Charge']
            charges.append(charge) \
            atom=Atom(f'{ASE_names[j][1][k][0]}', pos[k]) #the first element of the atom names gives the regular atom name 
            one_res_atoms.append(atom)
        one_res_Atoms=Atoms(one_res_atoms)
        one_res_Atoms.set_initial_charges(charges=charges)
        list_Atoms_objs.append(one_res_Atoms)
        one_res_atoms, one_res_Atoms =[], []
    return list_Atoms_objs #ASE(ASE_names, ASE_pos, dictt)

*^*^*^*^*^*^*^*^ alerta alerta! Ignore everything below  *^*^*^*^*^*^*^*^*^

In [ ]:
DF = dictt[dflist[0].index.name]#showing that: charge/type ~ f(residue type)
DF = DF.sort_values('Atom Type') #display(HTML(DF.to_html()))

In [ ]:
#display(HTML(dflist[5].to_html()))
dictt['158l-tyr-139-a'] #dictt[dflist[1].index.name] equivalent 

In [209]:
import sqlite3
import ase
from ase.db import connect
#create database 
db = ase.db.connect('test.db')

In [222]:
Atomsss=Atoms('Ag') #test writing
db.write(Atomsss)

2

In [58]:
#a = read('test.db')
from ase.io import read
a = read('/Users/jessihoernschemeyer/pKaSchNet/test.db')
a #WORKS

In [55]:
#write the atoms objects
#for i in range(9):
    #Atomss=list_Atoms_objs[i]
    #db.write(list_Atoms_objs[i])

In [234]:
for i in range(len(db)):
    db.delete(f'{i}')

In [237]:
row = db.get('C155N44O65S3') #QWORKSSS

In [238]:
db.get_atoms('C155N44O65S3') #WORKS

Atoms(symbols='C155N44O65S3', pbc=False)

In [231]:
from ase.io import read
a = read('/Users/jessihoernschemeyer/pKaSchNet/test.db')
a #WORKS

Atoms(symbols='Zn', pbc=False)

In [246]:
Atomsss=Atoms('Fe', tags = 1)
Atomsss

Atoms(symbols='Fe', pbc=False, tags=...)